# Poker strategy analysis

## Setup

In [ ]:
import numpy as np
import pandas as pd
pd.set_option('mode.chained_assignment',None)
import plotly.express as px
import plotly.graph_objects as go

In [ ]:
def set_stage_id(stage):
    if stage == 'Pre-Flop':
        return 1
    elif stage == 'Flop':
        return 2
    elif stage == 'Turn':
        return 3
    elif stage == 'River':
        return 4

def set_stage(stage_id):
    if stage_id == 1:
        return 'Pre-Flop'
    elif stage_id == 2:
        return 'Flop'
    elif stage_id == 3:
        return 'Turn'
    elif stage_id == 4:
        return 'River'

## Load data

In [ ]:
loggerFile = "C:\\Users\\user\\Documents\\Personal\\Analysis\\Poker\\poker_logger_20210107.xlsm"
game_id_list = None# [11, 12]

In [ ]:
# Read files
betting = pd.read_excel(loggerFile, sheet_name="betting")
results = pd.read_excel(loggerFile, sheet_name="results")

# Restrict to games in question
if game_id_list is not None:
    betting = betting.loc[betting['game_id'].isin(game_id_list)]
    results = results.loc[results['game_id'].isin(game_id_list)]

# Clean betting dataframe
betting['stage_id'] = betting['stage'].apply(set_stage_id)

# Calculate bet and pot amounts
bet_amounts = betting.groupby(['game_id', 'hand_id', 'player'])[['bet_amount']].sum()
pot_amounts = bet_amounts.groupby(['game_id', 'hand_id'])[['bet_amount']].sum()
pot_amounts = pot_amounts.rename(columns={'bet_amount': 'pot_amount'})

# Clean results dataframe
results['stage_id'] = results['stage'].apply(set_stage_id)
results['won_hand'] = np.where(results['outcome'].str.startswith('won'), 1, 
                               np.where(results['outcome'].str.startswith('split'), 0.5, 0))
results = pd.merge(results, bet_amounts, on=['game_id', 'hand_id', 'player'], how='left')
results = pd.merge(results, pot_amounts, on=['game_id', 'hand_id'], how='left')
results['net_winnings'] = results['won_hand'] * results['pot_amount']- results['bet_amount']

# Create hand summary dataframe
hands = results.loc[results['outcome'].str.startswith('won')]
hands = hands.rename(columns={'stage': 'win_stage', 
                              'stage_id': 'win_stage_id',
                              'player': 'winner', 
                              'hand': 'winner_hand',
                              'cards': 'winner_cards',
                              'player_position': 'winner_position'}).drop(columns=['won_hand'])

# Attach on the stage where each player folded, won or lost
stage_folded = betting.loc[betting['action'] == 'fold']
stage_folded = stage_folded.groupby(['game_id', 'hand_id', 'player'])[['stage_id']].min().reset_index()
stage_reached = pd.merge(stage_folded, results[['game_id', 'hand_id', 'player', 'stage_id']], 
                         on=['game_id', 'hand_id', 'player'], how='outer')
stage_reached['stage_id'] = np.minimum(stage_reached['stage_id_x'].fillna(99), stage_reached['stage_id_y'].fillna(99)).astype(int)
stage_reached = stage_reached[['game_id', 'hand_id', 'player', 'stage_id']].rename(columns={'stage_id': 'last_stage_id'})
results = pd.merge(results, stage_reached, on=['game_id', 'hand_id', 'player'])
results['saw_flop'] = np.where(results['last_stage_id'] > 1, 1, 0)
results = results.drop(columns=['stage', 'stage_id'])

#print('hands')
#display(hands.head())
#print('results')
#display(results.head())
#print('betting')
#display(betting.head())

## Pre-flop

In [ ]:
# Create aliases of the players with enough data to analyze
min_games = 5
pl = results.groupby(['game_id', 'player']).size().reset_index().groupby('player').size()
pl = pl[pl >= min_games]
players_list = pl.index

players_dict ={}
i = 64
for player in players_list:
    i += 1
    players_dict[player] = chr(i)


In [ ]:
def format_for_plot(df, players_dict):
    """Formats dataframe for scatter plot\
    """
    # Dictionaries for plots
    markercolor = {'A': 'blue', 'B': 'gray', 'C': 'red', 'D': 'green', 'E': 'purple', 'F': 'orange'}
    markerstyle = {'A': 'circle', 'B': 'square', 'C': 'diamond', 'D': 'cross', 'E': 'x', 'F': 'triangle-up'}
    
    df = df.loc[df['player'].isin(list(players_dict.keys()))]
    df['player_alias'] = df['player'].apply(lambda x: players_dict[x])
    df['marker_color'] = df['player_alias'].apply(lambda x: markercolor[x])
    df['marker_style'] = df['player_alias'].apply(lambda x: markerstyle[x])
    
    return df

def pre_flop_summary(betting, results, level, players_dict):
    """Creates summary of PFR and VPIP grouped at the desired level
    """

    # VPIP
    vpip = pd.pivot_table(results.loc[results['player_position'] >= 3],
                   index=level,
                   values=['saw_flop'],
                   aggfunc=['mean'])

    # PFR
    pre_flop = betting.loc[(betting['stage'] == 'Pre-Flop')]
    pre_flop['raise_ind'] = np.where(pre_flop['action'].isin(['raise']), 1, 0)
    agg = pre_flop.groupby(level + ['hand_id'])['raise_ind'].max().to_frame()
    agg = pd.merge(agg.reset_index(), results[level + ['hand_id', 'saw_flop']], on=level + ['hand_id'])
    agg = pd.pivot_table(agg, index=level, values=['saw_flop', 'raise_ind'], aggfunc=['sum'])
    agg['pfr'] = agg.loc[:, ('sum', 'raise_ind')] / agg.loc[:, ('sum', 'saw_flop')]

    # Join them together
    pre_flop_summ = pd.merge(vpip[['mean']], agg[['pfr']], on=level).reset_index()
    pre_flop_summ.columns = level + ['vpip', 'pfr']

    # Create fields for plot
    pre_flop_summ = format_for_plot(pre_flop_summ, players_dict)

    return pre_flop_summ

def post_flop_summary(betting, results, level, players_dict):
    """Creates summary of PFR and VPIP grouped at the desired level
    """
    
    # Stage summary
    stage_summary = pd.pivot_table(results, index=['game_id', 'player'], columns='last_stage_id', values='hand_id', aggfunc='count')

    # Summary of game/players being in play
    in_play = stage_summary.sort_index(axis=1, ascending=False).cumsum(axis=1).sort_index(axis=1)
    in_play = pd.melt(in_play.reset_index(), id_vars=['game_id', 'player'])
    in_play = in_play.rename(columns={'last_stage_id': 'stage_id', 'value': 'num_hands'})

    # Aggression calcs
    agg = betting.loc[(betting['stage_id'] >= 2) & (betting['action'].isin(['bet', 'raise']))]
    agg = agg.groupby(['game_id', 'player', 'stage_id'])[['hand_id']].count()
    agg = agg.rename(columns={'hand_id': 'num_aggs'})
    agg = pd.merge(in_play, agg, on=['game_id', 'player', 'stage_id'])
    agg = agg.groupby(level)[['num_hands', 'num_aggs']].sum()
    agg['agg_pc'] = agg['num_aggs'] / agg['num_hands']

    # Win percentages
    win_pc = results.loc[results['last_stage_id'] >= 2].groupby(level)[['won_hand']].mean()

    # Merge them together
    post_flop_summ = pd.merge(agg, win_pc, on=level).reset_index()
    
    # Create fields for plot
    post_flop_summ = format_for_plot(post_flop_summ, players_dict)
    
    return post_flop_summ


In [ ]:
player_game_pre = pre_flop_summary(betting, results, ['game_id', 'player'], players_dict)
player_pre = pre_flop_summary(betting, results, ['player'], players_dict)

player_game_post = post_flop_summary(betting, results, ['game_id', 'player'], players_dict)
player_post = post_flop_summary(betting, results, ['player'], players_dict)

In [ ]:
player_game_pre['label'] = 'Player ' + player_game_pre['player_alias'] + ', Game ' + player_game_pre['game_id'].astype(str)
    
fig = go.Figure()
fig.add_trace(go.Scatter(
    x=player_game_pre['vpip'],
    y=player_game_pre['pfr'],
    mode="markers+text",
    text=player_game_pre['player_alias'],
    hovertemplate=player_game_pre['label'],
    marker_color=player_game_pre['marker_color'],
    marker_symbol=player_game_pre['marker_style'],
    textposition="top center"
))

fig.update_layout(
    title="Pre-Flop Strategy by Player and Game",
    xaxis_title="Voluntarily Put $ in Pot % (VPIP)",
    yaxis_title="Pre-Flop Raise % (PFR)",
    xaxis_tickformat = '%',
    yaxis_tickformat = '%'
)
fig.show()

In [ ]:
fig.write_html('poker_pre_flop_scatter.html')

## Post-flop

In [ ]:
player_game_post['label'] = 'Player ' + player_game_post['player_alias'] + ', Game ' + player_game_post['game_id'].astype(str)
    
fig = go.Figure()
fig.add_trace(go.Scatter(
    x=player_game_post['agg_pc'],
    y=player_game_post['won_hand'],
    mode="markers+text",
    text=player_game_post['player_alias'],
    hovertemplate=player_game_post['label'],
    marker_color=player_game_post['marker_color'],
    marker_symbol=player_game_post['marker_style'],
    textposition="top center"
))

fig.update_layout(
    title="Post-Flop Strategy by Player and Game",
    xaxis_title="Post-flop Aggression Frequency % (Agg)",
    yaxis_title="Hands Won %",
    xaxis_tickformat = '%',
    yaxis_tickformat = '%'
)
fig.show()

In [ ]:
fig.write_html('poker_post_flop_scatter.html')

## Summary statistics

In [ ]:
player_metrics = pd.concat([player_pre[['player', 'player_alias', 'pfr', 'vpip']], player_post[['player_alias', 'agg_pc', 'won_hand']]], axis=1)

In [ ]:
results_in_scope = results.loc[results['game_id'].isin([ 1,  4,  6,  7,  8,  9, 10, 11, 12, 13, 14, 15])]
game_player_results = results_in_scope.groupby(['player'])[['net_winnings']].sum().reset_index()

In [ ]:
player_metrics.merge(game_player_results, on='player', how='left')